In [64]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from pathlib import Path

In [8]:
GDP_FILE_DIR = r"/Users/jamesmoro/Documents/Python/ClimateData/data/interim/Country_GDP_data.pkl"
GHG_2018_DF_FILE_DIR = r"/Users/jamesmoro/Documents/Python/ClimateData/data/interim/ghg_2018_dataframe.pkl"

In [9]:
gdp_df = pd.read_pickle(GDP_FILE_DIR)
ghg_2018_df = pd.read_pickle(GHG_2018_DF_FILE_DIR)

In [10]:
gdp_df.head()

2,Country Name,2018.0
3,Aruba,3.202189e+09
4,Africa Eastern and Southern,9.866107e+11
5,Afghanistan,1.835388e+10
6,Africa Western and Central,7.381313e+11
7,Angola,1.013532e+11


In [18]:
gdp_df.rename(columns = {"Country Name":"Country", 2018.0:"GDP"}, inplace = True)

In [19]:
gdp_df.head()

2,Country,GDP
3,Aruba,3.202189e+09
4,Africa Eastern and Southern,9.866107e+11
5,Afghanistan,1.835388e+10
6,Africa Western and Central,7.381313e+11
7,Angola,1.013532e+11


In [11]:
ghg_2018_df.head()

,Country,Year,CO2 Emissions,NF3 Emissions,CH4 Emissions,HFC Emissions,PFC Emissions,SF6 Emissions,N20 Emissions,Total
0,Australia,2018,415953.95,NaN,109532.21,11982.08,236.00,228.53,20114.44,558047.21
29,Austria,2018,66719.68,16.51,6438.63,1834.76,32.52,382.15,3526.08,78950.33
58,Belarus,2018,61871.68,NaN,16097.62,NaN,NaN,3.46,14019.84,91992.60
87,Belgium,2018,100207.84,0.65,7848.74,4469.84,131.32,95.08,5702.29,118455.76
116,Bulgaria,2018,43551.60,NaN,6753.67,2252.53,0.01,17.99,5239.79,57815.59


In [21]:
combined_df = pd.merge(ghg_2018_df, gdp_df, how = "outer", on = "Country")

In [22]:
combined_df.head()

,Country,Year,CO2 Emissions,NF3 Emissions,CH4 Emissions,HFC Emissions,PFC Emissions,SF6 Emissions,N20 Emissions,Total,GDP
0,Australia,2018.0,415953.95,NaN,109532.21,11982.08,236.00,228.53,20114.44,558047.21,1.432881e+12
1,Austria,2018.0,66719.68,16.51,6438.63,1834.76,32.52,382.15,3526.08,78950.33,4.548726e+11
2,Belarus,2018.0,61871.68,NaN,16097.62,NaN,NaN,3.46,14019.84,91992.60,6.003126e+10
3,Belgium,2018.0,100207.84,0.65,7848.74,4469.84,131.32,95.08,5702.29,118455.76,5.434107e+11
4,Bulgaria,2018.0,43551.60,NaN,6753.67,2252.53,0.01,17.99,5239.79,57815.59,6.623016e+10


In [23]:
combined_df["GDP"].isna().sum()

15

In [25]:

SAVE_DIR = r"/Users/jamesmoro/Documents/Python/ClimateData/data/interim"
file_path = Path(SAVE_DIR, "GHG_GDP_scatter_data.pkl")

pd.to_pickle(combined_df, file_path)

In [43]:
US_GDP = combined_df[combined_df["Country"] == "United States"]["GDP"]
US_GDP

254    2.061186e+13
Name: GDP, dtype: float64

# Plotting

In [79]:

fig = go.Figure(data=go.Scatter(x = combined_df["GDP"], 
y = combined_df["Total"], 
text = combined_df["Country"],
mode='markers',
marker_color=combined_df['GDP']))

fig.update_layout(title='GBP vs Total Greenhouse Gas Emissions', 
        xaxis_title = "GDP", 
        yaxis_title = "Total Emissions")
fig.update_xaxes(range = [0, 2.5e13])

trendline = px.scatter(combined_df, x = "GDP", y = "Total", trendline = "ols")
trendline = trendline.data[1]

fig.add_trace(go.Scatter(trendline,mode='lines', line=dict(color="red")))

fig.show()